In [1]:
import numpy as np
import torch
# 数据集处理，(地点,日期,最高气温)
import csv
origin_data = []
sta = '10001'
with open('Summary of Weather.csv','r',encoding='utf-8') as f:
    reader = csv.DictReader(f)
    for item in reader:
        if sta != item['STA']:
            break
        if (item['STA']!='' and item['Date']!='' and item['MaxTemp']!=''):
            origin_data.append(float(item['MaxTemp']))

days = len(origin_data)
series = torch.tensor(np.array(origin_data).reshape(1,days,1).astype(np.float32))

In [8]:
#预测1个值，前700天训练，后300天验证
X_train, y_train = series[:, :700], series[:, 701]
X_test, y_test = series[:, 700:1000], series[:, 1001]
X_train.shape,y_train.shape

(torch.Size([1, 700, 1]), torch.Size([1, 1]))

In [3]:

class RNN(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.rnn = torch.nn.LSTM(input_size=1, hidden_size=50, num_layers=1, batch_first=True)
    self.fc = torch.nn.Linear(50, 1)

  def forward(self, x):
    x, h = self.rnn(x) 
    y = self.fc(x[:,-1])
    return y

rnn = RNN()

In [10]:
from torch.utils.tensorboard import SummaryWriter
#训练模型
def train(model, train_data, train_target, test_data, test_target):
    writer = SummaryWriter()
    EPOCHS = 1000
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.MSELoss()
    for epoch in range(EPOCHS):
        model.train()
        output = model(train_data)
        loss = criterion(output, train_target)
        model.zero_grad()
        loss.backward()
        optimizer.step()
        model.eval()
    with torch.no_grad():
        for i,ele in enumerate(test_data[0]):
            writer.add_scalar("MaxTemp", ele.item(), i)
        # writer.add_scalar("MaxTemp", test_target.item(), len(test_data[0])) # 真实值
        output = model(test_data)
        writer.add_scalar("MaxTemp", output.item(), len(test_data[0])) # 预测值
        print(f'预测值：{output.item()}，真实值：{test_target.item()}')
    writer.close()

train(rnn,X_train, y_train,X_test, y_test)

预测值：31.66683578491211，真实值：33.33333206176758


In [11]:
#预测5个值，前700天训练，后300天验证
X_train, y_train = series[:, :700], series[:, 701:706].reshape(5)
X_test, y_test = series[:, 700:1000], series[:, 1001:1006].reshape(5)
X_train.shape,y_train

(torch.Size([1, 700, 1]),
 tensor([31.6667, 31.6667, 30.5556, 32.7778, 32.2222]))

In [6]:
class MULTI_RNN(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.rnn = torch.nn.LSTM(input_size=1, hidden_size=50, num_layers=1, batch_first=True)
    self.fc = torch.nn.Linear(50, 5)

  def forward(self, x):
    x, h = self.rnn(x) 
    y = self.fc(x[:,-1])
    return y.reshape(5)

rnn_multi = MULTI_RNN()

In [13]:
def train_multi(model, train_data, train_target, test_data, test_target):
    writer = SummaryWriter()
    EPOCHS = 1000
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.MSELoss()
    for epoch in range(EPOCHS):
        model.train()
        output = model(train_data)
        loss = criterion(output, train_target)
        model.zero_grad()
        loss.backward()
        optimizer.step()
        model.eval()
    with torch.no_grad():
        for i,ele in enumerate(test_data[0]):
            writer.add_scalar("MaxTemp_5", ele.item(), i)
        # 真实值
        # for i,ele in enumerate(test_target):
        #     writer.add_scalar("MaxTemp_5", ele.item(), len(test_data[0])+i) 
        # 预测值
        output = model(test_data)
        for i,ele in enumerate(output):
            writer.add_scalar("MaxTemp_5", ele.item(), len(test_data[0])+i) 
        print(f'预测值：{output}，真实值：{test_target}')
    writer.close()
train_multi(rnn_multi,X_train, y_train,X_test, y_test)

预测值：tensor([31.6667, 31.6667, 30.5556, 32.7779, 32.2223])，真实值：tensor([33.3333, 34.4444, 33.3333, 32.2222, 35.0000])
